In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import coint
from colorama import Fore, Style
import warnings
warnings.filterwarnings('ignore')

## 1. Data Loading

Load full cubic spline datasets for all metals.

In [2]:
# Define metals
metal_names = ['cobalt', 'copper', 'lithium', 'nickel']

# Load full datasets
print("="*60)
print("LOADING DATASETS")
print("="*60 + "\n")

metal_data = {}
for metal in metal_names:
    df = pd.read_csv(f'data/ALL_{metal}_prices_cubic_spline.csv', 
                     parse_dates=['Date'], index_col='Date')
    
    # Remove LISAME series if present
    if 'LISAME' in df.columns:
        df = df.drop(columns=['LISAME'])
        print(f"{metal.capitalize()}: LISAME excluded")
    
    metal_data[metal.capitalize()] = df
    print(f"{metal.capitalize()}: {df.shape[0]} rows, {df.shape[1]} series")
    print(f"  Columns: {df.columns.tolist()}")
    print()

LOADING DATASETS

Cobalt: 3876 rows, 6 series
  Columns: ['CODALY', 'COLMEX', 'COLMEA', 'COWUXI', 'COCOMX', 'COSMMS']

Copper: 2342 rows, 6 series
  Columns: ['CUDALY', 'CUCOMX', 'CULMEX', 'CUSMMG', 'CUSHFE', 'CUETFC']

Lithium: LISAME excluded
Lithium: 2202 rows, 6 series
  Columns: ['LIDALY', 'LICOMX', 'LILAMC', 'LIEALC', 'LIEABG', 'LILMEX']

Nickel: 2857 rows, 6 series
  Columns: ['NIDALY', 'NILMEX', 'NIETFN', 'NISHFE', 'NIWUXI', 'NIINDA']



## 2. Compute Log-Prices

Convert prices to log-prices for cointegration testing.

In [3]:
print("="*60)
print("COMPUTING LOG-PRICES")
print("="*60 + "\n")

metal_returns = {}
for metal in metal_names:
    df = metal_data[metal.capitalize()].copy()
    # Compute log-prices
    df_log_prices = np.log(df)
    metal_returns[metal.capitalize()] = df_log_prices
    print(f"{metal.capitalize()} log-prices: {df_log_prices.shape[0]} rows, {df_log_prices.shape[1]} series")

print("\nLog-prices will be used for cointegration analysis.")

COMPUTING LOG-PRICES

Cobalt log-prices: 3876 rows, 6 series
Copper log-prices: 2342 rows, 6 series
Lithium log-prices: 2202 rows, 6 series
Nickel log-prices: 2857 rows, 6 series

Log-prices will be used for cointegration analysis.


## 3. Configuration

Define series pairs and ticker mappings for each metal.

In [4]:
# Ticker mappings for LaTeX output (maps column names to display names)
ticker_mappings = {
    'Cobalt': {
        'CODALY': 'CODALY',
        'COLMEX': 'COLMEX',
        'COLME3': 'COLME3',
        'COLMEA': 'COLMEA',
        'COWUXI': 'COWUXI',
        'COCOMX': 'COCOMX',
        'COSMMS': 'COSMMS'
    },
    'Copper': {
        'CUDALY': 'CUDALY',
        'CULMEX': 'CULMEX',
        'CULME3': 'CULME3',
        'CULMEA': 'CULMEA',
        'CUWUXI': 'CUWUXI',
        'CUCOMX': 'CUCOMX',
        'CUSMMS': 'CUSMMS'
    },
    'Lithium': {
        'LIDALY': 'LIDALY',
        'LILMEX': 'LILMEX',
        'LILME3': 'LILME3',
        'LILMEA': 'LILMEA',
        'LIWUXI': 'LIWUXI',
        'LICOMX': 'LICOMX',
        'LISMMS': 'LISMMS'
    },
    'Nickel': {
        'NIDALY': 'NIDALY',
        'NILMEX': 'NILMEX',
        'NILME3': 'NILME3',
        'NILMEA': 'NILMEA',
        'NIWUXI': 'NIWUXI',
        'NICOMX': 'NICOMX',
        'NISMMS': 'NISMMS'
    }
}

# Define series pairs for testing (adjust based on actual column names)
# This is a template - will be constructed dynamically for each metal
def get_series_pairs(columns):
    """Generate all unique pairwise combinations of series."""
    pairs = []
    cols = [c for c in columns if c != 'LISAME']  # Exclude LISAME
    for i, col1 in enumerate(cols):
        for col2 in cols[i+1:]:
            pairs.append((col1, col2))
    return pairs

## 4. Helper Functions

In [5]:
def run_cointegration_tests(df, pairs, metal_name):
    """
    Run Engle-Granger cointegration tests for specified pairs.
    
    Parameters:
    - df: DataFrame with series as columns
    - pairs: List of (series1, series2) tuples
    - metal_name: Name of the metal for reporting
    
    Returns:
    - List of (series1, series2, score, p_value) tuples
    """
    results = []
    
    # Get list of all series involved
    all_series = set()
    for s1, s2 in pairs:
        all_series.add(s1)
        all_series.add(s2)
    
    # Drop rows with NaN in any of the required series
    df_clean = df[list(all_series)].dropna()
    
    print(f"\n{Fore.YELLOW}{metal_name} - Testing {len(pairs)} pairs{Style.RESET_ALL}")
    print(f"Data range: {df_clean.index.min()} to {df_clean.index.max()}")
    print(f"Observations: {len(df_clean)}\n")
    
    for series1, series2 in pairs:
        try:
            score, p_value, _ = coint(df_clean[series1], df_clean[series2])
            results.append((series1, series2, score, p_value))
        except Exception as e:
            print(f"Error testing {series1} vs {series2}: {e}")
            results.append((series1, series2, np.nan, np.nan))
    
    return results


def print_results_table(results, metal_name):
    """Print cointegration results as a formatted console table."""
    print(f"\n{Fore.YELLOW}{metal_name} Cointegration Test Results:{Style.RESET_ALL}")
    print(f"{Fore.CYAN}{'Series 1':<15}{'Series 2':<15}{'Score':<15}{'P-Value':<15}{Style.RESET_ALL}")
    
    for series1, series2, score, p_value in results:
        if pd.notna(p_value):
            p_value_color = Fore.GREEN if p_value < 0.05 else Style.RESET_ALL
            print(f"{series1:<15}{series2:<15}{Fore.GREEN}{score:<15.4f}{p_value_color}{p_value:<15.4f}{Style.RESET_ALL}")
        else:
            print(f"{series1:<15}{series2:<15}{'N/A':<15}{'N/A':<15}")


def print_latex_table(results, metal_name, ticker_map):
    """Generate LaTeX table for cointegration results."""
    print(f"\n{Fore.CYAN}LaTeX Table for {metal_name}:{Style.RESET_ALL}\n")
    
    print("\\begin{table}[ht]")
    print("\\centering")
    print("{\\small")
    print("\\begin{tabular}{llrr}")
    print("\\toprule")
    print("Series 1 & Series 2 & Score & P-Value \\\\")
    print("\\midrule")
    
    for series1, series2, score, p_value in results:
        if pd.notna(p_value):
            # Get ticker symbols from mapping
            ticker1 = ticker_map.get(series1, series1.replace("_", "\\_"))
            ticker2 = ticker_map.get(series2, series2.replace("_", "\\_"))
            
            # Format p-value with green color if significant
            p_value_str = f"\\textcolor{{green}}{{{p_value:.4f}}}" if p_value < 0.05 else f"{p_value:.4f}"
            
            print(f"{ticker1} & {ticker2} & {score:.4f} & {p_value_str} \\\\")
    
    print("\\bottomrule")
    print("\\end{tabular}")
    print("}")
    print(f"\\caption{{Cointegration Test Results - {metal_name}}}")
    print(f"\\label{{tab:cointegration_{metal_name.lower()}}}")
    print("\\end{table}")
    print()

## 5. Cobalt Cointegration Analysis (Log Prices)

In [6]:
metal = 'Cobalt'
df = metal_returns[metal]

# Generate all pairwise combinations
pairs = get_series_pairs(df.columns)

print(f"{'='*60}")
print(f"{metal.upper()} - COINTEGRATION ANALYSIS")
print(f"{'='*60}")

# Run tests
results = run_cointegration_tests(df, pairs, metal)

# Print results
print_results_table(results, metal)

# Print LaTeX table
print_latex_table(results, metal, ticker_mappings[metal])

COBALT - COINTEGRATION ANALYSIS

Cobalt - Testing 15 pairs
Data range: 2023-05-04 00:00:00 to 2025-07-21 00:00:00
Observations: 662


Cobalt Cointegration Test Results:
Series 1       Series 2       Score          P-Value        
CODALY         COLMEX         -4.8000        0.0004         
CODALY         COLMEA         -1.9932        0.5321         
CODALY         COWUXI         -4.2143        0.0035         
CODALY         COCOMX         -5.4731        0.0000         
CODALY         COSMMS         -2.7274        0.1899         
COLMEX         COLMEA         -2.1705        0.4396         
COLMEX         COWUXI         -4.5162        0.0012         
COLMEX         COCOMX         -5.2229        0.0001         
COLMEX         COSMMS         -2.9268        0.1286         
COLMEA         COWUXI         0.0104         0.9861         
COLMEA         COCOMX         -0.1947        0.9802         
COLMEA         COSMMS         -0.4973        0.9645         
COWUXI         COCOMX         -4.0764 

## 6. Copper Cointegration Analysis (Log Prices)

In [7]:
metal = 'Copper'
df = metal_returns[metal]

# Generate all pairwise combinations
pairs = get_series_pairs(df.columns)

print(f"{'='*60}")
print(f"{metal.upper()} - COINTEGRATION ANALYSIS")
print(f"{'='*60}")

# Run tests
results = run_cointegration_tests(df, pairs, metal)

# Print results
print_results_table(results, metal)

# Print LaTeX table
print_latex_table(results, metal, ticker_mappings[metal])

COPPER - COINTEGRATION ANALYSIS

Copper - Testing 15 pairs
Data range: 2024-03-06 00:00:00 to 2025-10-31 00:00:00
Observations: 432


Copper Cointegration Test Results:
Series 1       Series 2       Score          P-Value        
CUDALY         CUCOMX         -8.5209        0.0000         
CUDALY         CULMEX         -2.8382        0.1538         
CUDALY         CUSMMG         -3.3114        0.0532         
CUDALY         CUSHFE         -3.2345        0.0643         
CUDALY         CUETFC         -2.5930        0.2397         
CUCOMX         CULMEX         -1.8401        0.6101         
CUCOMX         CUSMMG         -3.4643        0.0357         
CUCOMX         CUSHFE         -3.3865        0.0439         
CUCOMX         CUETFC         -2.6937        0.2019         
CULMEX         CUSMMG         -3.1840        0.0726         
CULMEX         CUSHFE         -3.1878        0.0720         
CULMEX         CUETFC         -2.3124        0.3674         
CUSMMG         CUSHFE         -7.0362 

## 7. Lithium Cointegration Analysis (Log Prices)

In [8]:
metal = 'Lithium'
df = metal_returns[metal]

# Generate all pairwise combinations
pairs = get_series_pairs(df.columns)

print(f"{'='*60}")
print(f"{metal.upper()} - COINTEGRATION ANALYSIS")
print(f"{'='*60}")

# Run tests
results = run_cointegration_tests(df, pairs, metal)

# Print results
print_results_table(results, metal)

# Print LaTeX table
print_latex_table(results, metal, ticker_mappings[metal])

LITHIUM - COINTEGRATION ANALYSIS

Lithium - Testing 15 pairs
Data range: 2023-10-02 00:00:00 to 2025-10-31 00:00:00
Observations: 556


Lithium Cointegration Test Results:
Series 1       Series 2       Score          P-Value        
LIDALY         LICOMX         -2.8186        0.1599         
LIDALY         LILAMC         -3.2928        0.0557         
LIDALY         LIEALC         -1.9592        0.5497         
LIDALY         LIEABG         -1.9779        0.5400         
LIDALY         LILMEX         -2.7093        0.1962         
LICOMX         LILAMC         -2.2095        0.4195         
LICOMX         LIEALC         -2.0793        0.4872         
LICOMX         LIEABG         -2.4302        0.3107         
LICOMX         LILMEX         -6.2300        0.0000         
LILAMC         LIEALC         -2.7879        0.1696         
LILAMC         LIEABG         -2.7729        0.1745         
LILAMC         LILMEX         -2.5250        0.2682         
LIEALC         LIEABG         -5.05

## 8. Nickel Cointegration Analysis (Log Prices)

In [9]:
metal = 'Nickel'
df = metal_returns[metal]

# Generate all pairwise combinations
pairs = get_series_pairs(df.columns)

print(f"{'='*60}")
print(f"{metal.upper()} - COINTEGRATION ANALYSIS")
print(f"{'='*60}")

# Run tests
results = run_cointegration_tests(df, pairs, metal)

# Print results
print_results_table(results, metal)

# Print LaTeX table
print_latex_table(results, metal, ticker_mappings[metal])

NICKEL - COINTEGRATION ANALYSIS

Nickel - Testing 15 pairs
Data range: 2023-03-22 00:00:00 to 2025-07-21 00:00:00
Observations: 698


Nickel Cointegration Test Results:
Series 1       Series 2       Score          P-Value        
NIDALY         NILMEX         -7.9676        0.0000         
NIDALY         NIETFN         -3.2278        0.0654         
NIDALY         NISHFE         -2.7166        0.1937         
NIDALY         NIWUXI         -2.5521        0.2566         
NIDALY         NIINDA         -4.0748        0.0056         
NILMEX         NIETFN         -3.0892        0.0905         
NILMEX         NISHFE         -3.1959        0.0706         
NILMEX         NIWUXI         -3.1225        0.0839         
NILMEX         NIINDA         -4.9144        0.0002         
NIETFN         NISHFE         -2.1696        0.4401         
NIETFN         NIWUXI         -2.3781        0.3353         
NIETFN         NIINDA         -2.1094        0.4715         
NISHFE         NIWUXI         -9.1558 

## 9. Summary

**Methodology:**
- Engle-Granger cointegration test performed on log-prices
- Tests all unique pairwise combinations within each metal
- LISAME series excluded from analysis
- Log transformation applied to price levels: log(P_t)

**Interpretation:**
- **P-value < 0.05** (highlighted in green): Statistically significant cointegration at 5% level
- Lower p-values indicate stronger evidence of cointegration
- Cointegrated series tend to move together in the long run despite short-term deviations
- Testing on log-prices is appropriate for cointegration analysis as it stabilizes variance and handles exponential growth

**Note:** Cointegration is tested within each metal only, not across different metals.